In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from iprocessor import add_day_name_column, add_date_name_column, smooth_sundays_rolling_w7_l

#### the python script iprocessor, smoothes the data by implementing moving average

## there is need for 
- parameters
- initialization for the compartments

In [2]:
# Sample parameters,
contacts = 2.0
transmission_prob = 0.3649
reducing_transmission = 0.764
exposed_period = 5.2  #
asymptomatic_period = 7
infectious_period = 3.7
isolated_period = 11  # 11,23
prob_asymptomatic = 0.2
prob_quarant_inf = 0.05
test_asy = 0.171
dev_symp = 0.125
mortality_isolated = 0.002
mortality_infected = 0.01

In [3]:
total_population = 82_000_000  # Total number of individuals
E0 = 2026.25
A0 = 3798
I0 = 376.4
F0 = 2255
R0 = 170204
D0 = 9060
S0 = total_population - E0 - A0 - I0 - F0 - R0 - D0
initial_conditions = [S0, E0, A0, I0, F0, R0, D0]

### Dataframe

In [4]:
df = pd.read_csv(r'German_case_period_may_aug.csv')

In [5]:
#Convert 'Date' column to datetime format----------------------------------------------------------------------
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Modification------------------------------------------------------------------------------
# Add the 'days' column
df = add_day_name_column(df)
df = add_date_name_column(df)
# second modification with w7_l-----------------------------------------------------------------------------
df_observed = smooth_sundays_rolling_w7_l(df)
df_observed
# -----------------------------------------------------------------------------------------------------

,Date,Confirmed,Deaths,Recovered,n_confirmed,n_death,n_recovered,Infection_case,date_name,days,rolling_mean_r,rolling_mean_c,rolling_mean_d
0,2020-05-01,166468,8602,157866,428.333333,16.000000,412.333333,915,Friday,1,412.333333,428.333333,16.000000
1,2020-05-02,167160,8626,158534,529.250000,22.750000,506.500000,846,Saturday,2,506.500000,529.250000,22.750000
2,2020-05-03,167753,8650,159103,602.600000,25.800000,576.800000,805,Sunday,3,576.800000,602.600000,25.800000
3,2020-05-04,168585,8693,159892,654.666667,25.833333,628.833333,607,Monday,4,628.833333,654.666667,25.833333
4,2020-05-05,169481,8731,160750,682.000000,27.285714,654.714286,447,Tuesday,5,654.714286,682.000000,27.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2020-07-28,210338,9412,200926,679.285714,4.285714,675.000000,563,Tuesday,89,675.000000,679.285714,4.285714
89,2020-07-29,211114,9418,201696,694.000000,6.000000,688.000000,919,Wednesday,90,688.000000,694.000000,6.000000
90,2020-07-30,211868,9423,202445,709.285714,5.714286,703.571429,940,Thursday,91,703.571429,709.285714,5.714286
91,2020-07-31,212712,9438,203274,749.833333,6.000000,743.833333,996,Friday,92,743.833333,749.833333,6.000000


In [8]:
# Taking 'days' time column from dataframe
t_fit = np.array(df_observed['days'])
#tmax_base = len(t_fit_base)
tmax = len(t_fit)
array_dead = np.array(df_observed['n_death'])

### derivative equation for the flowchart

In [9]:
def derivative_rhs(x, X, contacts, transmission_prob, total_population, reducing_transmission,
                   exposed_period, asymptomatic_period, infectious_period, isolated_period,
                   prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    S, E, A, I, F, R, D = X
    derivS = - contacts * transmission_prob * S * (I + reducing_transmission * A) / total_population
    derivE = contacts * transmission_prob * S * (I + reducing_transmission * A) / total_population - E / exposed_period
    derivA = prob_asymptomatic * E / exposed_period - A / asymptomatic_period
    derivI = (
                         1 - prob_asymptomatic) * E / exposed_period + dev_symp * A / asymptomatic_period - I / infectious_period  # +
    derivF = prob_quarant_inf * I / infectious_period - F / isolated_period + test_asy * A / asymptomatic_period  # prob_isolated_asy*A/asymptomatic_period
    derivR = (1 - prob_quarant_inf - mortality_infected) * I / infectious_period + (
                1 - mortality_isolated) * F / isolated_period + (
                         1 - dev_symp - test_asy) * A / asymptomatic_period  # (1-prob_isolated_asy)*A / asymptomatic_period
    derivD = (mortality_infected) * I / infectious_period + mortality_isolated * F / isolated_period
    return np.array([derivS, derivE, derivA, derivI, derivF, derivR, derivD])

In [10]:
def seaifrd_model(x, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                  exposed_period, asymptomatic_period, infectious_period, isolated_period,
                  prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    def derivative(x, initial_conditions):
        return derivative_rhs(x, initial_conditions, contacts, transmission_prob, total_population,
                              reducing_transmission,
                              exposed_period, asymptomatic_period, infectious_period,
                              isolated_period, prob_asymptomatic,
                              prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    solution = solve_ivp(derivative, [0, tmax], initial_conditions, method='RK45')#t_eval=x[:,0]
    print(solution)
    return solution  # .y.flatten()

In [11]:
def objective_function_recoverd_dead(X,isolated_period):
    t = X

    # Model parameters (could be passed as arguments or fixed here)
    contacts = 2
    transmission_prob = 0.3
    total_population = 82000000
    reducing_transmission = 0.55
    exposed_period = 5.2
    asymptomatic_period = 7
    infectious_period = 3.7
    #isolated_period = 12
    prob_asymptomatic = 0.34
    prob_quarant_inf = 0.9303
    test_asy = 0.271
    dev_symp = 0.125
    mortality_isolated = 0.02
    mortality_infected = 0.1

    # Solve the model
    #sorted(set(t))
    solution = seaifrd_model(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                             exposed_period, asymptomatic_period, infectious_period, isolated_period,
                             prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    #print(solution)
    recovered = solution.y[5]
    
    return recovered


In [12]:
def objective_function_dead(X,isolated_period):
    t = X

    # Model parameters (could be passed as arguments or fixed here)
    contacts = 2
    transmission_prob = 0.3
    total_population = 82000000
    reducing_transmission = 0.55
    exposed_period = 5.2
    asymptomatic_period = 7
    infectious_period = 3.7
    #isolated_period = 12
    prob_asymptomatic = 0.34
    prob_quarant_inf = 0.9303
    test_asy = 0.271
    dev_symp = 0.125
    mortality_isolated = 0.02
    mortality_infected = 0.1

    # Solve the model
    #sorted(set(t))
    solution = seaifrd_model(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                             exposed_period, asymptomatic_period, infectious_period, isolated_period,
                             prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    #print(solution)
    dead = solution.y[6]
    

    return dead


In [13]:
### Calling curve_fit Function

In [15]:
t_end = df_observed['days'].iloc[-1]
print(f't_end = {(t_end)}, t_fit = {len(t_fit)}')
# Create a sequence from 0 to t_end
#t_fit = np.arange(0, 2*tmax, 1)

popt, pcov = curve_fit(objective_function_dead, t_fit, array_dead)


t_end = 93, t_fit = 93
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.171e-01 ...  9.054e+01  9.300e+01]
        y: [[ 8.181e+07  8.181e+07 ...  3.316e+07  2.953e+07]
            [ 2.026e+03  2.152e+03 ...  8.821e+06  8.368e+06]
            ...
            [ 1.702e+05  1.705e+05 ...  2.790e+07  3.161e+07]
            [ 9.060e+03  9.066e+03 ...  2.785e+06  3.135e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 200
     njev: 0
      nlu: 0


ValueError: operands could not be broadcast together with shapes (32,) (93,) 

In [27]:
popt

NameError: name 'popt' is not defined

In [ ]:
perr_r_d = np.sqrt(np.diag(pcov))
perr_r_d

In [ ]:
### regarding for the curve_fitting; do we need to find the first derivation for the data for the dead and recovered before 
#  we apply curve fit?